In [ ]:
import pandas               as pd
import plotly.express       as px
import plotly.graph_objects as go
import spacy

In [ ]:
file_path = "data\lingqs.csv"
clean_data_file_path = "data\clean.csv"
nlp = spacy.load("de_core_news_lg")

df = pd.read_csv(file_path, header=0)

df.head()

## Data Cleaning
Deleting useless column (such as 68 tags and meaninglanguageN).

In [ ]:
for i in range(1, 69) :
    col = "tag"+ format(i)
    df.drop([col], axis=1, inplace=True)

Replacing all languages except ["en", "uk", "ru"]

In [ ]:
df["meaninglanguage1"] = df["meaninglanguage1"].fillna("en")
all_langs = df["meaninglanguage1"].unique()
amb_langs = []

for l in all_langs :
    if not(l == "en" or l == "uk" or l == "ru") : amb_langs.append(l)

df.loc[df["meaninglanguage1"].isin(amb_langs)]

for l in amb_langs :
    df["meaninglanguage1"] = df["meaninglanguage1"].replace(l, "en")

df["meaninglanguage1"].unique()

Dropping all rows, where `"phrase"` or `"meaning1"` is `NaN`

In [ ]:
df = df.dropna(subset=["phrase", "meaning1"], axis=0)
df = df.reset_index(drop=True)

df.info()

In [ ]:
df.to_csv(path_or_buf=clean_data_file_path, index=False)

analizing words i have here and finding lemmas, so that "geh", "geht" and "gehen" is the same one

In [ ]:
# Using DataFrame.insert() to add a new column
new_col = []

for i in df.index : new_col.append("")

df.insert(1, "lemma", new_col, True)

In [ ]:
for i in df.index :
    doc = nlp(df.at[i, "term"])
    df.at[i, "lemma"] = doc[0].lemma_

In [ ]:
df.head()

In [ ]:
df["lemma"].unique().__len__()

In [ ]:
# transforming data for better understanding
df1 = df[["term", "lemma"]].groupby("lemma").count()
df1

In [ ]:
px.histogram(df1, x="term")

Let's try to analyse in phrases

In [ ]:
# data = {"term":[]}
lemmas = []

# doc = nlp(df.at[0, "phrase"])

for i in df.index :
    doc = nlp(df.at[i, "phrase"])
    for token in doc:
        lemmas.append(token.lemma_)

In [ ]:
df2 = pd.DataFrame(lemmas, columns=["lemma"])
df2.info()

In [ ]:
# transforming data for better understanding
df2["lemma_freq"] = df2.groupby("lemma")["lemma"].transform("count")
df2.head()

In [ ]:
pd.qcut(df2["lemma_freq"], q=10)

In [ ]:
px.histogram(df2, x="lemma_freq")